In [8]:
import logging
logging.getLogger().setLevel(logging.INFO)
import mxnet as mx
import numpy as np

fname = mx.test_utils.download('http://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data')
data = np.genfromtxt(fname, delimiter=',')[:,1:]
label = np.array([ord(l.split(',')[0])-ord('A') for l in open(fname, 'r')])

batch_size = 32
ntrain = int(data.shape[0]*0.8)
train_iter = mx.io.NDArrayIter(data[:ntrain, :], label[:ntrain], batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(data[ntrain:, :], label[ntrain:], batch_size)

INFO:root:letter-recognition.data exists, skipping download


In [9]:
#from gen_input import gen_input
##这个数据还不太好使，我换别的数据试试
#data_iter = gen_input()
#data_iter.reset()
#data = data_iter.next()
#data_features = data.data[0]
#data_label=data.label[0]
#train_iter = mx.io.NDArrayIter(data_features,data_label,batch_size=6,shuffle=True)
#valid_iter = train_iter

In [10]:
net = mx.sym.Variable('data')
net = mx.sym.FullyConnected(net, name='fc1', num_hidden=64)
net = mx.sym.Activation(net, name='relu1', act_type="relu")
net = mx.sym.FullyConnected(net, name='fc2', num_hidden=26)
#net = mx.sym.FullyConnected(net, name='fc3', num_hidden=2)
net = mx.sym.SoftmaxOutput(net, name='softmax')
#mx.viz.plot_network(net)

In [11]:
mod = mx.mod.Module(symbol=net,
                    context=mx.cpu(),
                    data_names=['data'],
                    label_names=['softmax_label'])

In [12]:
# allocate memory given the input data and label shapes
mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
# initialize parameters by uniform random numbers
mod.init_params(initializer=mx.init.Uniform(scale=.1))
# use SGD with learning rate 0.1 to train
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.1), ))
# use accuracy as the metric
metric = mx.metric.create('acc')
# train 5 epochs, i.e. going over the data iter one pass
for epoch in range(5):
    train_iter.reset()
    metric.reset()
    for batch in train_iter:
        mod.forward(batch, is_train=True)       # compute predictions
        mod.update_metric(metric, batch.label)  # accumulate prediction accuracy
        mod.backward()                          # compute gradients
        mod.update()                            # update parameters
    print('Epoch %d, Training %s' % (epoch, metric.get()))

Epoch 0, Training ('accuracy', 0.44274999999999998)
Epoch 1, Training ('accuracy', 0.65031249999999996)
Epoch 2, Training ('accuracy', 0.70787500000000003)
Epoch 3, Training ('accuracy', 0.74306249999999996)
Epoch 4, Training ('accuracy', 0.76231249999999995)


In [14]:
train_iter.provide_data[0]

DataDesc[data,(32, 16),<class 'numpy.float32'>,NCHW]

In [17]:
train_iter.label

[('softmax_label', 
  [  2.   3.   1. ...,  14.  25.   5.]
  <NDArray 16000 @cpu(0)>)]